In [1]:
#librerias estandar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\train.csv", index_col="ID")
#data.drop(columns=["F_TIENEINTERNET.1"], inplace=True)  
data.head()

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
ID,,,,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


**Nota aclaratoria**

El preprocesado con promedios hace referencia a la estrategia de reducción de categorias de la variable PRGM_ACADEMICO. En este caso la reducción se hizo abarcando en la misma categoria aquellos programas cuyos promedios fueran similares. Esto con la intención de lograr una reducción de dimensionalidad mayor al método del umbral

In [17]:
#preprocesado

#tratamiento de variable objetivo
orden = ["bajo", "medio-bajo", "medio-alto", "alto"]
data['RENDIMIENTO_GLOBAL'] = pd.Categorical(data['RENDIMIENTO_GLOBAL'], ordered=True, categories=orden)
data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].cat.codes  # convierte a 0,1,2,3

#Nulos
#True si el registro no tiene ningun valor nulo, False en caso contrario
data["DATOS_COMPLETOS"] = data.isna().sum(axis=1) == 0
data = data.fillna("sin_dato") #rellenar nulos

#columna repetida
data = data.drop(columns=["F_TIENEINTERNET.1"])

#reducción de categorias de E_PRGM_ACADEMICO usando promedios de rendimiento

#media del rendimiento por programa
media_rend = data.groupby('E_PRGM_ACADEMICO')['RENDIMIENTO_GLOBAL'].mean()
#40 grupos ordenados por rendimiento
clusters = pd.qcut(media_rend, q=50, labels=False)
programa_to_cluster = clusters.to_dict() #para luego aplicar al test.csv
# clusters
data['E_PRGM_ACADEMICO_AGRUPADO'] = data['E_PRGM_ACADEMICO'].map(clusters)
data.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)


#binarias
binarias = [
    "F_TIENEINTERNET",
    "F_TIENELAVADORA",
    "F_TIENEAUTOMOVIL",
    "E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR",
    "E_PRIVADO_LIBERTAD"
]

for col in binarias:
    data[col] = (
        data[col]
        .astype(str)
        .str.strip()
        .str.upper()
        .replace({
            "SI": 1, "S": 1,
            "NO": 0, "N": 0,
            "SIN_DATO": -1
        })
    )

#ordinales
# Codificación para E_VALORMATRICULAUNIVERSIDAD
map_matricula = {
    "Menos de 500 mil": 1,
    "Entre 500 mil y menos de 1 millón": 2,
    "Entre 1 millón y menos de 2.5 millones": 3,
    "Entre 2.5 millones y menos de 4 millones": 4,
    "Entre 4 millones y menos de 5.5 millones": 5,
    "Entre 5.5 millones y menos de 7 millones": 6,
    "Más de 7 millones": 7,
    "No pagó matrícula": 0,
    "sin_dato": -1
}
data["E_VALORMATRICULAUNIVERSIDAD"] = data["E_VALORMATRICULAUNIVERSIDAD"].map(map_matricula)

# Codificación para F_ESTRATOVIVIENDA
map_estrato = {
    "Estrato 1": 1,
    "Estrato 2": 2,
    "Estrato 3": 3,
    "Estrato 4": 4,
    "Estrato 5": 5,
    "Estrato 6": 6,
    "Sin Estrato": 0,
    "sin_dato": -1
}
data["F_ESTRATOVIVIENDA"] = data["F_ESTRATOVIVIENDA"].map(map_estrato)

# Codificación para E_HORASSEMANATRABAJA
map_horas = {
    "0": 0,   # por si aparece como string
    0: 0,     # por si aparece como número
    "Menos de 10 horas": 1,
    "Entre 11 y 20 horas": 2,
    "Entre 21 y 30 horas": 3,
    "Más de 30 horas": 4,
    "sin_dato": -1
}
data["E_HORASSEMANATRABAJA"] = data["E_HORASSEMANATRABAJA"].map(map_horas)

educacion_map = {
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Técnica o tecnológica incompleta": 5,
    "Técnica o tecnológica completa": 6,
    "Educación profesional incompleta": 7,
    "Educación profesional completa": 8,
    "Postgrado": 9,
    
    # categorías sin información → -1
    "sin_dato": -1,
    "No sabe": -1,
    "No Aplica": -1
}

data["F_EDUCACIONPADRE"] = data["F_EDUCACIONPADRE"].map(educacion_map)
data["F_EDUCACIONMADRE"] = data["F_EDUCACIONMADRE"].map(educacion_map)

#one hot
data = pd.get_dummies(data, columns=["E_PRGM_DEPARTAMENTO"], prefix="DEP", drop_first=False)
data = pd.get_dummies(data, columns=["E_PRGM_ACADEMICO_AGRUPADO"], prefix="PRGM", drop_first=False)

#periodo academico
data["PERIODO_ACADEMICO"] = data["PERIODO_ACADEMICO"].astype(str)

data["ANIO"] = data["PERIODO_ACADEMICO"].str[:4].astype(int)
data["PERIODO"] = data["PERIODO_ACADEMICO"].str[-1].astype(int)

data.drop(columns=["PERIODO_ACADEMICO"], inplace=True)

#numericas
cols_norm = ["INDICADOR_1", "INDICADOR_2", "INDICADOR_3", "INDICADOR_4"]

# 3. Crear scaler y ajustarlo
scaler = StandardScaler()
data[cols_norm] = scaler.fit_transform(data[cols_norm])

#data.drop(columns=["E_PRGM_ACADEMICO_AGRUPADO"], inplace=True)
#data.drop(columns=["E_PRGM_DEPARTAMENTO"], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_6632\2734516202.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_6632\2734516202.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_6632\2734516202.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `p

In [18]:
data.head()

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,PRGM_42,PRGM_43,PRGM_44,PRGM_45,PRGM_46,PRGM_47,PRGM_48,PRGM_49,ANIO,PERIODO
ID,,,,,,,,,,,,,,,,,,,,,
904256,6,1,3,1,5,1,1,0,0,1,...,False,False,False,False,False,False,False,False,2021,2
645256,4,0,3,0,6,1,0,0,0,1,...,False,False,False,False,False,False,False,False,2021,2
308367,4,4,3,1,4,1,0,0,0,0,...,False,False,False,False,False,False,False,False,2020,3
470353,5,0,4,1,-1,1,0,0,0,1,...,False,False,False,False,False,False,False,False,2019,5
989032,4,3,3,1,2,1,1,0,0,1,...,False,False,False,False,False,False,False,False,2021,2


In [26]:
#división entrenamiento y testeo
from sklearn.model_selection import train_test_split

X = data.drop(columns=["RENDIMIENTO_GLOBAL"])
y = data["RENDIMIENTO_GLOBAL"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y  # mantiene la proporción de clases
)

In [10]:
#librerias del modelo
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report


Agrego un paso extra de preprocesamiento para trabajar mejor con el svm (standardScaler) y PCA

In [18]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="rbf", C=1, gamma="scale"))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average="weighted"))
print(classification_report(y_test, y_pred))

Accuracy: 0.39060649819494586
F1 Score: 0.37716577129362233
              precision    recall  f1-score   support

           0       0.39      0.51      0.44     34597
           1       0.32      0.29      0.30     34455
           2       0.29      0.18      0.23     34324
           3       0.50      0.57      0.53     35124

    accuracy                           0.39    138500
   macro avg       0.37      0.39      0.38    138500
weighted avg       0.38      0.39      0.38    138500



Entrenamiento se tarda demasiado, entonces intentar con submuestreo


Nota: Intento bastante implementar el svm porque a mi parecer es el que mejor debería funcionar para esta tarea

In [8]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="linear", C=1, gamma="scale"))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average="weighted"))
print(classification_report(y_test, y_pred))

Accuracy: 0.3628953068592058
F1 Score: 0.3436103316008508
              precision    recall  f1-score   support

           0       0.37      0.47      0.41     34597
           1       0.30      0.28      0.29     34455
           2       0.28      0.13      0.18     34324
           3       0.43      0.57      0.49     35124

    accuracy                           0.36    138500
   macro avg       0.34      0.36      0.34    138500
weighted avg       0.35      0.36      0.34    138500



In [27]:
#Modelo de prueba xgboost
# =========================================================
# Entrenamiento de XGBoost
# =========================================================
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Convertir a DMatrix opcional, pero podemos usar directamente sklearn API
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # clasificación multiclase
    num_class=4,                # número de clases
    n_estimators=200,           # número de árboles
    max_depth=6,                # profundidad máxima de cada árbol
    learning_rate=0.1,          # tasa de aprendizaje
    subsample=0.8,              # para regularización
    colsample_bytree=0.8,       # para regularización
    random_state=42,
    n_jobs=-1                   # usar todos los cores disponibles
)

# Entrenamiento
xgb_model.fit(X_train, y_train)

# Predicción
y_pred = xgb_model.predict(X_test)

# Evaluación
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")
print(classification_report(y_test, y_pred))


Accuracy: 0.4339061371841155
F1 Score: 0.4214925549697347
              precision    recall  f1-score   support

           0       0.46      0.57      0.51     51896
           1       0.33      0.27      0.30     51682
           2       0.33      0.25      0.29     51486
           3       0.55      0.63      0.59     52686

    accuracy                           0.43    207750
   macro avg       0.42      0.43      0.42    207750
weighted avg       0.42      0.43      0.42    207750



abajo fit con datos de test

In [28]:
data = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\test.csv", index_col="ID")
data.head()

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
ID,,,,,,,,,,,,,,,,,,,
550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [29]:
#preprocesado

#tratamiento de variable objetivo
#orden = ["bajo", "medio-bajo", "medio-alto", "alto"]
##data['RENDIMIENTO_GLOBAL'] = pd.Categorical(data['RENDIMIENTO_GLOBAL'], ordered=True, categories=orden)
#data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].cat.codes  # convierte a 0,1,2,3

#Nulos
#True si el registro no tiene ningun valor nulo, False en caso contrario
data["DATOS_COMPLETOS"] = data.isna().sum(axis=1) == 0
data = data.fillna("sin_dato") #rellenar nulos

#columna repetida
data = data.drop(columns=["F_TIENEINTERNET.1"])



#binarias
binarias = [
    "F_TIENEINTERNET",
    "F_TIENELAVADORA",
    "F_TIENEAUTOMOVIL",
    "E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR",
    "E_PRIVADO_LIBERTAD"
]

for col in binarias:
    data[col] = (
        data[col]
        .astype(str)
        .str.strip()
        .str.upper()
        .replace({
            "SI": 1, "S": 1,
            "NO": 0, "N": 0,
            "SIN_DATO": -1
        })
    )

#ordinales
# Codificación para E_VALORMATRICULAUNIVERSIDAD
map_matricula = {
    "Menos de 500 mil": 1,
    "Entre 500 mil y menos de 1 millón": 2,
    "Entre 1 millón y menos de 2.5 millones": 3,
    "Entre 2.5 millones y menos de 4 millones": 4,
    "Entre 4 millones y menos de 5.5 millones": 5,
    "Entre 5.5 millones y menos de 7 millones": 6,
    "Más de 7 millones": 7,
    "No pagó matrícula": 0,
    "sin_dato": -1
}
data["E_VALORMATRICULAUNIVERSIDAD"] = data["E_VALORMATRICULAUNIVERSIDAD"].map(map_matricula)

# Codificación para F_ESTRATOVIVIENDA
map_estrato = {
    "Estrato 1": 1,
    "Estrato 2": 2,
    "Estrato 3": 3,
    "Estrato 4": 4,
    "Estrato 5": 5,
    "Estrato 6": 6,
    "Sin Estrato": 0,
    "sin_dato": -1
}
data["F_ESTRATOVIVIENDA"] = data["F_ESTRATOVIVIENDA"].map(map_estrato)

# Codificación para E_HORASSEMANATRABAJA
map_horas = {
    "0": 0,   # por si aparece como string
    0: 0,     # por si aparece como número
    "Menos de 10 horas": 1,
    "Entre 11 y 20 horas": 2,
    "Entre 21 y 30 horas": 3,
    "Más de 30 horas": 4,
    "sin_dato": -1
}
data["E_HORASSEMANATRABAJA"] = data["E_HORASSEMANATRABAJA"].map(map_horas)

educacion_map = {
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Técnica o tecnológica incompleta": 5,
    "Técnica o tecnológica completa": 6,
    "Educación profesional incompleta": 7,
    "Educación profesional completa": 8,
    "Postgrado": 9,
    
    # categorías sin información → -1
    "sin_dato": -1,
    "No sabe": -1,
    "No Aplica": -1
}

data["F_EDUCACIONPADRE"] = data["F_EDUCACIONPADRE"].map(educacion_map)
data["F_EDUCACIONMADRE"] = data["F_EDUCACIONMADRE"].map(educacion_map)

#one hot
data = pd.get_dummies(data, columns=["E_PRGM_DEPARTAMENTO"], prefix="DEP", drop_first=False)

#periodo academico
data["PERIODO_ACADEMICO"] = data["PERIODO_ACADEMICO"].astype(str)

data["ANIO"] = data["PERIODO_ACADEMICO"].str[:4].astype(int)
data["PERIODO"] = data["PERIODO_ACADEMICO"].str[-1].astype(int)

data.drop(columns=["PERIODO_ACADEMICO"], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_6632\1954454178.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_6632\1954454178.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_6632\1954454178.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `p

In [30]:
# -----------------------------
# Preprocesamiento de data (test)
# -----------------------------

# Mapear E_PRGM_ACADEMICO a clusters según train
data['E_PRGM_ACADEMICO_AGRUPADO'] = data['E_PRGM_ACADEMICO'].map(programa_to_cluster)

# Asignar -1 a programas no vistos en train y convertir a int
data['E_PRGM_ACADEMICO_AGRUPADO'] = data['E_PRGM_ACADEMICO_AGRUPADO'].fillna(-1).astype(int)

# Eliminar columna original
data.drop(columns=['E_PRGM_ACADEMICO'], inplace=True)

# One-hot encoding
data = pd.get_dummies(data, columns=['E_PRGM_ACADEMICO_AGRUPADO'])

# Alinear columnas con las que vio el modelo durante train
for c in X_train.columns:
    if c not in data.columns:
        data[c] = 0

# Reordenar columnas igual que en X_train
data = data[X_train.columns]

# Ahora se puede predecir con el modelo entrenado
y_pred = xgb_model.predict(data)

In [31]:
# Diccionario de decodificación
decode_dict = {0: "bajo", 1: "medio-bajo", 2: "medio-alto", 3: "alto"}

# Decodificar las predicciones
y_pred_decoded = [decode_dict[p] for p in y_pred]

# Crear DataFrame para exportar
output = pd.DataFrame({
    "ID": data.index,   # o data["ID"] si ID es columna
    "RENDIMIENTO_GLOBAL": y_pred_decoded
})

# Guardar a CSV
output.to_csv("predicciones_testxg2.csv", index=False)